In [3]:
import numpy as np
import h5py
import sys
import glob

sys.path.append('/data/wiay/2297403c/amaze_model_select/AMAZE_model_selection/plot/flow_results/')
from plot_functions import load_result_samps

channels = ['CE', 'CHE', 'GC', 'NSC', 'SMT']

cont_dir='/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/outputs/cont_GWTC3/prod_retrainedCE/'
disc_dir='/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/outputs/discrete_GWTC3/flow_retrainedCE/'
KDE_dir='/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/outputs/discrete_GWTC3/KDEs/'
det_dir='/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/plots/prod_091224/data/cont_retrainedCE_detectable_betas.hdf5'
result_files={'cont':glob.glob(f'{cont_dir}/*.hdf5'),\
        'disc':glob.glob(f'{disc_dir}/*.hdf5'),\
        'KDE':glob.glob(f'{KDE_dir}/*.hdf5'),\
        'det':glob.glob(f'{det_dir}')}

#initialise mean branching fractions and their upper and lower quantiles
betas={'cont':dict.fromkeys(channels, 'val'),\
        'disc':dict.fromkeys(channels, 'val'),\
        'KDE':dict.fromkeys(channels, 'val')}
betasup={'cont':dict.fromkeys(channels, 'val'),\
        'disc':dict.fromkeys(channels, 'val'),\
        'KDE':dict.fromkeys(channels, 'val')}
betaslow={'cont':dict.fromkeys(channels, 'val'),\
        'disc':dict.fromkeys(channels, 'val'),\
        'KDE':dict.fromkeys(channels, 'val')}

betas_det=dict.fromkeys(channels, 'val')
betas_detup=dict.fromkeys(channels, 'val')
betas_detlow=dict.fromkeys(channels, 'val')

#chi_b, alpha_CE median and quantiles for continuous inference
chib_median={'cont':'val',\
        'disc':'val',\
        'KDE':'val'}
chib_up={'cont':'val',\
        'disc':'val',\
        'KDE':'val'}
chib_low={'cont':'val',\
        'disc':'val',\
        'KDE':'val'}

alphaCE_median={'cont':'val',\
        'disc':'val',\
        'KDE':'val'}
alphaCE_up={'cont':'val',\
        'disc':'val',\
        'KDE':'val'}
alphaCE_low={'cont':'val',\
        'disc':'val',\
        'KDE':'val'}

BF_chi10={'disc':'val',\
        'KDE':'val'}
BF_alphaCE52={'disc':'val',\
        'KDE':'val'}
        
#bayes factors between chi_b and alpha_CE models for KDE and flows?

# KL divergences between KDE and flow
KL_diff = dict.fromkeys(channels, 'val')


#sigfig formatting
title_fmt=".2g"
fmt = "{{0:{0}}}".format(title_fmt).format

In [2]:
#calculate macros for branching fractions and chib, alphaCE
for inference in ['cont', 'disc', 'KDE']:
    samps = load_result_samps(result_files[inference])
    beta_samps = samps[:,2:]
    for cidx, channel in enumerate(channels):
        betas[inference][channel] = fmt(np.median(beta_samps[:,cidx]))
        betasup[inference][channel] = fmt(np.quantile(beta_samps[:,cidx], .95)-np.median(beta_samps[:,cidx]))
        betaslow[inference][channel] = fmt(np.median(beta_samps[:,cidx])-np.quantile(beta_samps[:,cidx],.05))

    chib_samps = samps[:,0]
    alphaCE_samps = samps[:,1]

    chib_median[inference] = fmt(np.median(chib_samps))
    chib_up[inference] = fmt(np.quantile(chib_samps, .95)-np.median(chib_samps))
    chib_low[inference] = fmt(np.median(chib_samps)-np.quantile(chib_samps,.05))

    alphaCE_median[inference] = fmt(np.median(alphaCE_samps))
    alphaCE_up[inference] = fmt(np.quantile(alphaCE_samps, .95)-np.median(alphaCE_samps))
    alphaCE_low[inference] = fmt(np.median(alphaCE_samps)-np.quantile(alphaCE_samps,.05))

    if inference in ['disc', 'KDE']:
        BF_chi10[inference] = fmt(len(chib_samps[chib_samps==1])/len(chib_samps[chib_samps==0]))
        BF_alphaCE52[inference] = fmt(len(alphaCE_samps[alphaCE_samps==4])/len(alphaCE_samps[alphaCE_samps==3]))


In [9]:
#calculate macros for detectable branching fractions
det_samps = load_result_samps(result_files['det'], detectable=True)
beta_det_samps = det_samps[:,2:]
for cidx, channel in enumerate(channels):
    betas_det[channel] = fmt(np.median(beta_det_samps[:,cidx]))
    betas_detup[channel] = fmt(np.quantile(beta_det_samps[:,cidx], .95)-np.median(beta_det_samps[:,cidx]))
    betas_detlow[channel] = fmt(np.median(beta_det_samps[:,cidx])-np.quantile(beta_det_samps[:,cidx],.05))


In [7]:
#calculate macros for mean KL difference
for channel in channels:
    flow_KL= np.load(f'/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/plots/prod_091224/data/{channel}_flow_KL.npy')
    kde_KL= np.load(f'/data/wiay/2297403c/amaze_model_select/Nflows_AMAZE_paper/plots/prod_091224/data/{channel}_KDE_KL.npy')
    KL_diff[channel]=fmt(np.mean(flow_KL-kde_KL))

In [18]:
#write macros to file
#need rounding?
with open('result_macros.sty', 'w') as macros_file:
    for channel in channels:
        for inference in ['cont', 'disc', 'KDE']:
            command = '{\Beta'+channel+inference+'}{'+str(betas[inference][channel])+'}'
            macros_file.write(f'\\newcommand{command}\n')
            command = '{\Beta'+channel+inference+'up}{'+str(betasup[inference][channel])+'}'
            macros_file.write(f'\\newcommand{command}\n')
            command = '{\Beta'+channel+inference+'low}{'+str(betaslow[inference][channel])+'}'
            macros_file.write(f'\\newcommand{command}\n')

        command = '{\KLdiff'+channel+'}{'+str(KL_diff[channel])+'}'
        macros_file.write(f'\\newcommand{command}\n')
    
    for inference in ['cont', 'disc', 'KDE']:
        command = '{\chib'+inference+'}{'+str(chib_median[inference])+'}'
        macros_file.write(f'\\newcommand{command}\n')
        command = '{\chib'+inference+'up}{'+str(chib_up[inference])+'}'
        macros_file.write(f'\\newcommand{command}\n')
        command = '{\chib'+inference+'low}{'+str(chib_low[inference])+'}'
        macros_file.write(f'\\newcommand{command}\n')

        command = '{\\alphaCE'+inference+'}{'+str(alphaCE_median[inference])+'}'
        macros_file.write(f'\\newcommand{command}\n')
        command = '{\\alphaCE'+inference+'up}{'+str(alphaCE_up[inference])+'}'
        macros_file.write(f'\\newcommand{command}\n')
        command = '{\\alphaCE'+inference+'low}{'+str(alphaCE_low[inference])+'}'
        macros_file.write(f'\\newcommand{command}\n')
    
    for inference in ['disc', 'KDE']:
        command = '{\BF_chi10'+inference+'}{'+str(BF_chi10[inference])+'}'
        macros_file.write(f'\\newcommand{command}\n')
        command = '{\BF_alphaCE52'+inference+'}{'+str(BF_alphaCE52[inference])+'}'
        macros_file.write(f'\\newcommand{command}\n')

In [7]:
#write whole command containing median, upper and lower quantiles
with open('result_macros.sty', 'w') as macros_file:
    for channel in channels:
        for inference in ['cont', 'disc', 'KDE']:
            command = '{\Beta'+channel+inference+'all}{'+str(betas[inference][channel])+'^{+'+str(betasup[inference][channel])+'}_{-'+str(betaslow[inference][channel])+'}'+'}'
            macros_file.write(f'\\newcommand{command}\n')

    for inference in ['cont', 'disc', 'KDE']:
        command = '{\chib'+inference+'all}{'+str(chib_median[inference])+'^{+'+str(chib_up[inference])+'}_{-'+str(chib_low[inference])+'}'+'}'
        macros_file.write(f'\\newcommand{command}\n')

        command = '{\\alphaCE'+inference+'all}{'+str(alphaCE_median[inference])+'^{+'+str(alphaCE_up[inference])+'}_{-'+str(alphaCE_low[inference])+'}'+'}'
        macros_file.write(f'\\newcommand{command}\n')

In [14]:
#whole command for beta dets
#calculate macros for detectable branching fractions - percents
det_samps = load_result_samps(result_files['det'], detectable=True)
beta_det_samps = det_samps[:,2:]
for cidx, channel in enumerate(channels):
    betas_det[channel] = fmt(np.median(beta_det_samps[:,cidx])*100)
    betas_detup[channel] = fmt((np.quantile(beta_det_samps[:,cidx], .95)-np.median(beta_det_samps[:,cidx]))*100)
    betas_detlow[channel] = fmt((np.median(beta_det_samps[:,cidx])-np.quantile(beta_det_samps[:,cidx],.05))*100)

with open('result_macros.sty', 'w') as macros_file:
    for channel in channels:
        command = '{\BetaDetPerc'+channel+'all}{'+str(betas_det[channel])+'^{+'+str(betas_detup[channel])+'}_{-'+str(betas_detlow[channel])+'}'+'}'
        macros_file.write(f'\\newcommand{command}\n')